In [183]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tmdbsimple as tmdb
import os
import re
import sys
import pickle
import time
%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('../src'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [184]:
with open('../Data/api_key.pkl', 'rb') as hnd:
    tmdb.API_KEY = pickle.load(hnd)['api_key']

In [185]:
data = pd.read_csv('../Data/NetflixViewingHistory.csv')
import gather_data as eda1
netflix_df = eda1.parse_netflix(data)

Total number of TV Show + Movies:  1405
TV Show vs Movie
Dataframe shape:  (1405, 6)


In [186]:
shows = netflix_df[netflix_df['TV_Show_flag'] == 'TV_Show']
movies = netflix_df[netflix_df['TV_Show_flag'] == 'Movie']

In [187]:
search= tmdb.Search()

## Begin looking into TV Shows 

The plan is to find the name and season of the show and find the match in tmdb. We can get the names of all the episodes in the season and find the TMDB id number from there. After we have the id number, we query using the tmdb tv search and get the run time and genre. There's plenty of info on the way  

Steps: 
1. Find TV Show name and season 
2. Find episode match in Season API call 
3. Get the ID from the season API 
4. Query using TV to get the run time + other info 

In [188]:
shows.shape

(1357, 6)

In [189]:
shows.head()

,Title,Date,Show Name,Season,Episode Name,TV_Show_flag
0,Dating Around: Season 1: Gurki,2019-03-03,Dating Around,Season 1,Gurki,TV_Show
1,Dating Around: Season 1: Luke,2019-03-03,Dating Around,Season 1,Luke,TV_Show
2,The Umbrella Academy: Season 1: We Only See Ea...,2019-02-19,The Umbrella Academy,Season 1,We Only See Each Other at Weddings and Funerals,TV_Show
4,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV_Show
5,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV_Show


In [190]:
temp_season_numb = shows['Season'].str.split('Season ', n=2, expand=True)[1]

In [191]:
# Fix this regex 
shows["Season #2"] = temp_season_numb.str.extract('^(\d{0,2}$)?')

/home/owner/Documents/Github_projects/Netflix_TMDB/venv1/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [192]:
shows["Season #2"].value_counts()

1     318
3     189
2     186
4     171
5     146
6     108
7      78
8      40
9      24
10     17
Name: Season #2, dtype: int64

In [193]:
count_per_tv_title = shows.groupby('Show Name').size().sort_values(ascending=False)
count_per_tv_title.head()

Show Name
Friends                 171
30 Rock                 134
The Office (U.S.)       115
Parks and Recreation     90
White Collar             81
dtype: int64

In [194]:
dups = shows.duplicated()
dups.sum()

0

In [195]:
unique_tv_show = shows['Show Name'].unique()
show_name1 = unique_tv_show[0]
print(show_name1)

Dating Around


In [196]:
res1 = search.tv(query=show_name1)['results'][0]

In [197]:
res1_id = res1['id']

In [198]:
tv_show_ids = {}
for show_name in unique_tv_show:
    search_result = search.tv(query=show_name)
    if len(search_result['results']) >= 1:
        temp_id = search_result['results'][0]['id']
        tv_show_ids[show_name]=temp_id
        time.sleep(0.3)

In [199]:
len(unique_tv_show)

74

In [200]:
tv_show_ids_df = pd.Series(tv_show_ids, name='TMDB_id')

In [201]:
shows2 = pd.merge(shows, tv_show_ids_df, how='left', left_on='Show Name', right_index=True)

In [202]:
shows2.head()

,Title,Date,Show Name,Season,Episode Name,TV_Show_flag,Season #2,TMDB_id
0,Dating Around: Season 1: Gurki,2019-03-03,Dating Around,Season 1,Gurki,TV_Show,1,86807
1,Dating Around: Season 1: Luke,2019-03-03,Dating Around,Season 1,Luke,TV_Show,1,86807
2,The Umbrella Academy: Season 1: We Only See Ea...,2019-02-19,The Umbrella Academy,Season 1,We Only See Each Other at Weddings and Funerals,TV_Show,1,75006
4,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV_Show,1,73126
5,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV_Show,1,73126


In [203]:
tv_res1 = tmdb.TV_Seasons(res1_id, season_number=1).info()
tv_res1

{'_id': '5c6524ac0e0a2625d197ba24',
 'air_date': '2019-02-14',
 'episodes': [{'air_date': '2019-02-14',
   'episode_number': 1,
   'id': 1707145,
   'name': 'Luke',
   'overview': "He may be a ladies' man, but Luke is ready to settle down -- after some light drinking, deep kissing and one date's awkward-but-adorable door snafu.",
   'production_code': '',
   'season_number': 1,
   'show_id': 86807,
   'still_path': None,
   'vote_average': 0.0,
   'vote_count': 0,
   'crew': [],
   'guest_stars': []},
  {'air_date': '2019-02-14',
   'episode_number': 2,
   'id': 1707146,
   'name': 'Gurki',
   'overview': 'Fiercely independent Gurki is eager to open up about divorce, until one date radically challenges her candor. Will she back down or speak her mind?',
   'production_code': '',
   'season_number': 1,
   'show_id': 86807,
   'still_path': None,
   'vote_average': 0.0,
   'vote_count': 0,
   'crew': [],
   'guest_stars': []},
  {'air_date': '2019-02-14',
   'episode_number': 3,
   'id':

## Begin GB

In [204]:
show_season_gb = shows2.groupby(['TMDB_id', 'Season #2'])

In [205]:
show_season_gb.groups

{(52,
  '7'): Int64Index([1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086], dtype='int64'),
 (211, '1'): Int64Index([1179, 1180], dtype='int64'),
 (578, '1'): Int64Index([1306], dtype='int64'),
 (578, '4'): Int64Index([1224], dtype='int64'),
 (615, '1'): Int64Index([561, 562, 563], dtype='int64'),
 (709, '1'): Int64Index([1371, 1372], dtype='int64'),
 (900,
  nan): Int64Index([1221, 1222, 1223, 1228, 1229, 1230, 1238, 1239, 1240, 1241], dtype='int64'),
 (1044, '1'): Int64Index([912], dtype='int64'),
 (1063, nan): Int64Index([1186, 1187, 1188, 1204, 1205], dtype='int64'),
 (1395,
  '5'): Int64Index([1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173,
             1174, 1175],
            dtype='int64'),
 (1395, '6'): Int64Index([1160, 1161, 1162], dtype='int64'),
 (1403, '3'): Int64Index([431], dtype='int64'),
 (1405,
  '1'): Int64Index([1311, 1312, 1313, 1314, 1315, 1317, 1318, 1319, 1320, 1321, 1324,
             1327],
            dtype='int64'),
 (1405

In [244]:
group_name = (73126, '1')

In [245]:
demo_season_gb = show_season_gb.get_group(group_name)

In [246]:
demo_season_gb

,Title,Date,Show Name,Season,Episode Name,TV_Show_flag,Season #2,TMDB_id
4,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV_Show,1,73126
5,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV_Show,1,73126
6,American Vandal: Season 1: Hard Facts: Vandali...,2018-11-18,American Vandal,Season 1,Hard Facts: Vandalism and Vulgarity,TV_Show,1,73126


In [247]:
tv_res1 = tmdb.TV_Seasons(int(group_name[0]), season_number=int(group_name[1])).info()

In [248]:
tv_res1

{'_id': '5983c73792514158a3004927',
 'air_date': '2017-09-15',
 'episodes': [{'air_date': '2017-09-15',
   'episode_number': 1,
   'id': 1353037,
   'name': 'Hard Facts: Vandalism and Vulgarity',
   'overview': "When senior class clown Dylan Maxwell is expelled for vandalism, sophomore Peter Maldonado begins an investigation that questions Dylan's guilt.",
   'production_code': None,
   'season_number': 1,
   'show_id': 73126,
   'still_path': '/zFBPk6HVwGP7gli68ADFVlKZqUg.jpg',
   'vote_average': 8.333,
   'vote_count': 3,
   'crew': [],
   'guest_stars': [{'id': 82626,
     'name': 'Calum Worthy',
     'credit_id': '5c0e061f0e0a2638bc0b92a2',
     'character': 'Alex Trimboli',
     'order': 2,
     'gender': 2,
     'profile_path': '/6evKNNeSDzdno2d1BfwH7vMG9hy.jpg'}]},
  {'air_date': '2017-09-15',
   'episode_number': 2,
   'id': 1353038,
   'name': 'A Limp Alibi',
   'overview': "Key witness Alex Trimboli's account of the incident comes under fire when other students doubt his boas

In [249]:
l1 = demo_season_gb['Episode Name'].str.strip().tolist()

In [250]:
l1

['Nailed', 'A Limp Alibi', 'Hard Facts: Vandalism and Vulgarity']

In [251]:
episodes = {episode['name']:(episode['id'], episode['episode_number']
                             , episode['vote_average'], episode['vote_count'], episode['air_date'],  episode['overview']) 
            for episode in tv_res1['episodes']
            if episode['name'] in l1}

In [252]:
temp_episode_df = pd.DataFrame.from_dict(episodes, orient='index', columns = ['id', 'episode_number', 'vote_average', 'vote_count', 'air_date', 'overview'])

In [253]:
temp_episode_df

,id,episode_number,vote_average,vote_count,air_date,overview
Hard Facts: Vandalism and Vulgarity,1353037,1,8.333,3,2017-09-15,When senior class clown Dylan Maxwell is expel...
A Limp Alibi,1353038,2,8.333,3,2017-09-15,Key witness Alex Trimboli's account of the inc...
Nailed,1353039,3,8.333,3,2017-09-15,"Peter scrutinizes the motives of Ms. Shapiro, ..."


In [254]:
bigger_temp_df = pd.merge(demo_season_gb, temp_episode_df, how='left', left_on='Episode Name', right_index=True)

In [255]:
bigger_temp_df.head()

,Title,Date,Show Name,Season,Episode Name,TV_Show_flag,Season #2,TMDB_id,id,episode_number,vote_average,vote_count,air_date,overview
4,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV_Show,1,73126,1353039,3,8.333,3,2017-09-15,"Peter scrutinizes the motives of Ms. Shapiro, ..."
5,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV_Show,1,73126,1353038,2,8.333,3,2017-09-15,Key witness Alex Trimboli's account of the inc...
6,American Vandal: Season 1: Hard Facts: Vandali...,2018-11-18,American Vandal,Season 1,Hard Facts: Vandalism and Vulgarity,TV_Show,1,73126,1353037,1,8.333,3,2017-09-15,When senior class clown Dylan Maxwell is expel...


In [265]:
# tv_query = tmdb.TV(id=int(bigger_temp_df.iloc[1]['id']))
tv_query = tmdb.TV(2004)

In [266]:
tv_query.info()

{'backdrop_path': '/8VmOlxllljk6AiamMK8odBFNKl5.jpg',
 'created_by': [{'id': 40629,
   'credit_id': '52571ece19c295711410ce28',
   'name': 'Linwood Boomer',
   'gender': 2,
   'profile_path': None}],
 'episode_run_time': [30, 22],
 'first_air_date': '2000-01-09',
 'genres': [{'id': 35, 'name': 'Comedy'}],
 'homepage': '',
 'id': 2004,
 'in_production': False,
 'languages': ['en'],
 'last_air_date': '2006-05-14',
 'last_episode_to_air': {'air_date': '2006-05-14',
  'episode_number': 22,
  'id': 137016,
  'name': 'Graduation',
  'overview': "Malcolm struggles with his speech when he's elected valedictorian of his graduating class at school. However, Lois refuses to listen to any of his complaints and reveals that she wants him to become President of the United States. Meanwhile, Reese does everything he can to get a permanent job as a high school janitor, which includes creating the most disgusting mess ever. In the meantime, Hal has problems with Malcolm's Harvard tuition, while Francis

In [258]:
demo_season_gb

,Title,Date,Show Name,Season,Episode Name,TV_Show_flag,Season #2,TMDB_id
4,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV_Show,1,73126
5,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV_Show,1,73126
6,American Vandal: Season 1: Hard Facts: Vandali...,2018-11-18,American Vandal,Season 1,Hard Facts: Vandalism and Vulgarity,TV_Show,1,73126


In [219]:
shows.columns

Index(['Title', 'Date', 'Show Name', 'Season', 'Episode Name', 'TV_Show_flag',
       'Season #2'],
      dtype='object')